In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_absolute_error , mean_squared_error , r2_score 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LinearRegression, Ridge

df= pd.read_excel(r"C:\Users\HP\Documents\DATASET\ENB2012_data.xlsx")
column_names = { 'X1' : 'Relative_Compactness' , 'X2' : 'Surface_Area' ,
                'X3' : 'Wall_Area' , 'X4' : 'Roof_Area' , 'X5' : 'Overall_Height' ,
                'X6' : 'Orientation' , 'X7' : 'Glazing_Area' , 
                'X8' : 'Glazing_Area_Distribution' , 
                'Y1' : 'Heating_Load' , 'Y2' : 'Cooling_Load' } 
df = df.rename(columns=column_names)
df.head(5)

,Relative_Compactness,Surface_Area,Wall_Area,Roof_Area,Overall_Height,Orientation,Glazing_Area,Glazing_Area_Distribution,Heating_Load,Cooling_Load
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [7]:
scaler = MinMaxScaler() 
normalised_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns) 
features_df = normalised_df.drop(columns=[ 'Heating_Load' , 'Cooling_Load' ]) 
heating_target = normalised_df[ 'Heating_Load' ] 
 
x_train, x_test, y_train, y_test = train_test_split(features_df, heating_target, test_size= 0.3 , random_state= 1 )

In [18]:
linear_model = LinearRegression() 
linear_model.fit(x_train, y_train) 
linear_predicted_values = linear_model.predict(x_test) 

lmae = mean_absolute_error(y_test, linear_predicted_values) 
lrss = np.sum(np.square(y_test - linear_predicted_values))  
lrmse = np.sqrt(mean_squared_error(y_test, linear_predicted_values))  
lr2_score = r2_score(y_test, linear_predicted_values) 

print ("The mean absolute error is : ", round(lmae, 3 ))
print("The residual sum of squafres is : ", round(lrss, 3 ))
print("The mean squared error is: ", round(lrmse, 3 ))
print("The r squared error is: ", round(lr2_score, 3 ))

The mean absolute error is :  0.063
The residual sum of squafres is :  1.823
The mean squared error is:  0.089
The r squared error is:  0.893


In [16]:
ridge_reg = Ridge(alpha= 0.5 )
ridge_reg.fit(x_train, y_train)
predicted_values = linear_model.predict(x_test) 

ridmae = mean_absolute_error(y_test, predicted_values)  
ridrss = np.sum(np.square(y_test - predicted_values)) 
ridrmse = np.sqrt(mean_squared_error(y_test, predicted_values))
ridr2_score = r2_score(y_test, predicted_values)


print ("The mean absolute error is : ", round(ridmae, 3 ))
print("The residual sum of squares is : ", round(ridrss, 3 ))
print("The mean squared error is: ", round(ridrmse, 3 ))
print("The r squared error is: ", round(ridr2_score, 3 ))

The mean absolute error is :  0.063
The residual sum of squares is :  1.823
The mean squared error is:  0.089
The r squared error is:  0.893


In [17]:
lasso_reg = Lasso(alpha= 0.001 )
lasso_reg.fit(x_train, y_train)
lamae = mean_absolute_error(y_test, linear_predicted_values) 
larss = np.sum(np.square(y_test - linear_predicted_values))  
larmse = np.sqrt(mean_squared_error(y_test, linear_predicted_values))  
lar2_score = r2_score(y_test, linear_predicted_values) 

print ("The mean absolute error is : ", round(lamae, 3 ))
print("The residual sum of squares is : ", round(larss, 3 ))
print("The mean squared error is: ", round(larmse, 3 ))
print("The r squared error is: ", round(lar2_score, 3 ))

The mean absolute error is :  0.063
The residual sum of squares is :  1.823
The mean squared error is:  0.089
The r squared error is:  0.893


In [23]:
def get_weights_df(model, feat, col_name) :  
    weights = pd.Series(model.coef_, feat.columns).sort_values() 
    weights_df = pd.DataFrame(weights).reset_index() 
    weights_df.columns = [ 'Features' , col_name] 
    weights_df[col_name].round( 3 )
    return weights_df 
linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight' ) 
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight' ) 
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight' )
final_weights = pd.merge(linear_model_weights, ridge_weights_df, on= 'Features' )
final_weights = pd.merge(final_weights, lasso_weights_df, on= 'Features' )

print(final_weights)

                    Features  Linear_Model_Weight  Ridge_Weight  Lasso_weight
0               Surface_Area        -3.458933e+12     -0.062275      0.000000
1       Relative_Compactness        -6.064125e-01     -0.283471     -0.027719
2                Orientation         5.081177e-03      0.003369      0.000000
3  Glazing_Area_Distribution         3.077054e-02      0.029088      0.021431
4               Glazing_Area         2.125349e-01      0.212449      0.206132
5             Overall_Height         3.967285e-01      0.442467      0.463482
6                  Wall_Area         2.017711e+12      0.103061      0.200087
7                  Roof_Area         2.594200e+12     -0.163192     -0.000000
